In [ ]:
import sys
import termate
import nafigator
import stanza
import os
import logging

In [ ]:
logging.basicConfig(stream=sys.stdout,
                    format='%(asctime)s %(message)s',
                    level=logging.INFO)
nlp = {"nl": stanza.Pipeline("nl", processors='tokenize,pos', verbose=False), 
       "en": stanza.Pipeline("en", processors='tokenize,pos', verbose=False)}

In [ ]:
# read the IATE file
IATE_FILE = os.path.join("P:", "projects", "tbx-data", "iate", "IATE_export_small.tbx")
ref = termate.TbxDocument().open(IATE_FILE)
# read the lassy file
LASSY_FILE = os.path.join("P:", "projects", "tbx-data", "lassy", "lassy_with_insurance.tbx")
lassy = termate.TbxDocument().open(LASSY_FILE)

In [ ]:
def merge_terms_dict(a, b):
    for key in b.keys():
        if key in a.keys() and a[key]['dc:language']==b[key]['dc:language']:
            for item in b[key]['dc:uri']:
                a[key]['dc:uri'].append(item)
            for item in b[key]['frequency']:
                a[key]['frequency'].append(item)
        else:
            a[key] = {}
            for b_key in b[key].keys():
                a[key][b_key] = b[key][b_key]

In [ ]:
# Create an empty TermBase
t = termate.TbxDocument()
t.generate(params = {"sourceDesc": "TBX file, created via dnb/termate"})

In [ ]:
# termNotes = {"normativeAuthorization": "regulatedTerm", 
#              "authoritative body": "European Union"}
termNotes = {}

names = ['Solvency 2 Directive',
         'Solvency 2 Delegated Acts', 
         'European Climate Law', 
         'Sustainable Finance Disclosure Regulation']

# create terms dictionary of subset of languages
terms = {}

for name in names:
    for language in ['NL', 'EN', 'DE', 'FR']:
        DOC_FILE = os.path.join("P:", "projects", "naf-data", "data", "legislation", name, name+" - "+language+".naf.xml")
        doc = nafigator.NafDocument().open(DOC_FILE)
        if doc.language=="nl":
            nafigator.glue_terms_separated_by_soft_hyphens(doc, "nl", nlp)
            nafigator.glue_terms_separated_by_hard_hyphens(doc, "nl", nlp)
        merge_terms_dict(terms, nafigator.extract_terms(doc, termNotes = termNotes))

In [ ]:
name = "legislation"

In [ ]:
t.create_tbx_from_terms_dict(terms=terms)
# t.write(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))

In [ ]:
t.add_termnotes_from_tbx(reference=lassy, params={'number_of_word_components': 5})
# t.write(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))

In [ ]:
t.copy_from_tbx(reference=ref)
t.write(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))

In [ ]:
# t = termate.TbxDocument().open(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))
# t.validate(validation_file=os.path.join("..", "data", "dialects", "TBX-DNB.rng")

In [ ]:
t = termate.TbxDocument().open(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))
t.to_excel(os.path.join("P:", "projects", "tbx-data", "termbases", name+".xlsx"), languages=["en", "nl"])